## First Sample Model
- it produced to deploy Yolo Nano on 0.5 core in each container
- it does not have complete details of the environment

## Heuristic

In [1]:
import itertools

In [4]:
def evaluate_configuration(vm_current_details, container_details, vm_predicted_utilization, container_cores, container_models, model_accuracy, vm_max_util):
    best_config = {}
    best_score = float('-inf')

    container_configs = list(itertools.product(container_cores, container_models))
    num_containers = len(container_details.keys())

    # Iterate over all possible configurations
    for configs in itertools.product(container_configs, repeat=num_containers):
        total_accuracy = 0
        total_cost = 0
        total_sla_violation = 0
        valid = True

        # Deep copy of VM details to simulate placements
        current_vm_details = {k: list(v) for k, v in vm_current_details.items()}
        
        for container_id, (core, model) in zip(container_details.keys(), configs):
            vm_id = container_details[container_id][3]
            current_vm_details[vm_id][2] += core  # Update used CPU
            if current_vm_details[vm_id][2] > current_vm_details[vm_id][3]:  # Check if capacity is exceeded
                valid = False
                break

            total_accuracy += model_accuracy[model]
            total_cost += core
            total_sla_violation += container_details[container_id][2]  # Assuming SLA violation rate doesn't change
            
            # Update the predicted utilization (percentage)
            core_count = current_vm_details[vm_id][0]
            predicted_util = vm_predicted_utilization[vm_id][1]
            max_util = vm_max_util[core_count]
            
            for i in range(6):
                predicted_util[i] += (core / core_count) * 100
                if predicted_util[i] > max_util:
                    valid = False
                    break

            if not valid:
                break

        if valid:
            score = total_accuracy - total_cost - total_sla_violation
            if score > best_score:
                best_score = score
                best_config = configs

    return best_config

vm_current_details = {
    0: (2, 31.91, 0.616, 1.93, 1.314), 
    1: (4, 79.08, 3.1, 3.92, 0.82),
    2: (6, 70.68, 4.1, 5.8, 1.7)
}

container_details = {
    0: ('medium', 0.5, 1, 0),
    1: ('small', 1, 0.8, 2),
    2: ('small', 0.5, 0.2, 1),
    3: ('medium', 2, 1, 2)
}

vm_predicted_utilization = {
    0: [56.758137, 56.95575, 56.832123, 56.850445, 56.78143, 56.77969],
    1: [43.508926, 48.362938, 43.35024, 47.931683, 43.576538, 47.833996],
    2: [38.18736, 38.50672, 38.527256, 38.652214, 38.768574, 38.929024]
}

model_accuracy = {'nano': 45.7, 'small': 56.8, 'medium': 64.1}
# each container can have one selected core from below given pool
container_cores = [0.5, 1, 2]
# each container can have one selected model from below given list 
container_models = ['nano', 'small', 'medium']

vm_max_util = {
    2: 80, 4:80, 6:80
}

# Calculate best configuration
best_config = evaluate_configuration(vm_current_details, container_details, vm_predicted_utilization, container_cores, container_models, model_accuracy, vm_max_util)

TypeError: 'float' object is not subscriptable

In [8]:
from pulp import *

prob = pulp.LpProblem("ContainerPlacement", LpMaximize)

# Indices and sets
num_vms = 3
num_containers = 4
cores = [0.5, 1, 2]
models = ['nano', 'small', 'medium']
model_accuracy = {'nano': 45.7, 'small': 56.8, 'medium': 64.1}

# Variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_containers) for j in range(num_vms)), cat='Binary')
y = pulp.LpVariable.dicts("y", ((i, k) for i in range(num_containers) for k in cores), cat='Binary')
z = pulp.LpVariable.dicts("z", ((i, m) for i in range(num_containers) for m in models), cat='Binary')
u = pulp.LpVariable.dicts("u", ((i, j, k) for i in range(num_containers) for j in range(num_vms) for k in cores), cat='Binary')

# Objective Function
prob += pulp.lpSum(model_accuracy[m] * z[(i, m)] for i in range(num_containers) for m in models) - \
        pulp.lpSum(y[(i, k)] * k for i in range(num_containers) for k in cores) - \
        pulp.lpSum(container_details[i][2] for i in range(num_containers))

# Constraints
# Each container must be placed on exactly one VM
for i in range(num_containers):
    prob += pulp.lpSum(x[(i, j)] for j in range(num_vms)) == 1

# Each container must have exactly one core size
for i in range(num_containers):
    prob += pulp.lpSum(y[(i, k)] for k in cores) == 1

# Each container must have exactly one model version
for i in range(num_containers):
    prob += pulp.lpSum(z[(i, m)] for m in models) == 1

# Ensure u_ijk is correctly representing the use of core k on VM j for container i
for i in range(num_containers):
    for j in range(num_vms):
        for k in cores:
            prob += u[(i, j, k)] >= x[(i, j)] + y[(i, k)] - 1
            prob += u[(i, j, k)] <= x[(i, j)]
            prob += u[(i, j, k)] <= y[(i, k)]

# VM capacity constraints
for j in range(num_vms):
    prob += pulp.lpSum(u[(i, j, k)] * k for i in range(num_containers) for k in cores) + vm_current_details[j][2] <= vm_current_details[j][3]

# VM utilization constraints for future timestamps
for j in range(num_vms):
    for t in range(6):
        prob += pulp.lpSum(u[(i, j, k)] * (k / vm_current_details[j][0]) * 100 for i in range(num_containers) for k in cores) + \
                vm_predicted_utilization[j][1][t] <= vm_max_util[vm_current_details[j][0]]

# Solve the problem
prob.solve()

# Output results
print("Status:", pulp.LpStatus[prob.status])

# Display the best configuration
for i in range(num_containers):
    for j in range(num_vms):
        if pulp.value(x[(i, j)]) == 1:
            print(f"Container {i} is placed on VM {j}")
    for k in cores:
        for j in range(num_vms):
            if pulp.value(u[(i, j, k)]) == 1:
                print(f"Container {i} uses {k} cores on VM {j}")
    for m in models:
        if pulp.value(z[(i, m)]) == 1:
            print(f"Container {i} uses model {m}")

TypeError: 'float' object is not subscriptable

In [ ]:
print("Best Configuration:")
for container_id, (core, model) in zip(container_details.keys(), best_config):
    print(f"Container {container_id}: Model={model}, Cores={core}")

In [ ]:
# Define your parameters
accuracy = {'nano': 45.7, 'small': 56.8, 'medium': 64.1}
cost = {0.5: 0.5, 1: 1, 2: 2}
predicted_loads = {
    2: [56.758137, 56.95575, 56.832123, 56.850445, 56.78143, 56.77969],
    4: [43.508926, 48.362938, 43.35024, 47.931683, 43.576538, 47.833996],
    6: [38.18736, 38.50672, 38.527256, 38.652214, 38.768574, 38.929024]
}
current_loads = {
    2: 75.15, 4: 43.03, 6: 55.04
}
sla_violation_rate = {
    ('nano', 0.5): 0.05,
    ('small', 1): 0.09,
    ('small', 0.5): 0.3,
    ('medium', 2): 0
}
num_tasks = len(predicted_loads)
nodes = 3  # Total number of nodes in the cluster
max_load_percentage = 0.8

# Create a new model
model = pulp.LpProblem("edge_computing", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("x", [(i, k, n) for i in accuracy.keys() for k in cost.keys() for n in range(nodes)], 0, 1, pulp.LpBinary)

# Decision variables
accuracy_levels = ['nano', 'small', 'medium']
cost_levels = [0.5, 1, 2]

# Binary variables for each task, accuracy level, and cost level
task_accuracy_cost = pulp.LpVariable.dicts("task_accuracy_cost",
                                           ((task, accuracy, cost) for task in predicted_loads for accuracy in accuracy_levels for cost in cost_levels),
                                           cat='Binary')

# Continuous variables for predicted loads on each node
node_loads = pulp.LpVariable.dicts("node_load", current_loads.keys(), lowBound=0, cat='Continuous')

# Objective function (example: maximize performance by minimizing SLA violations)
model += pulp.lpSum(task_accuracy_cost[(task, accuracy, cost)] * (1 - sla_violation_rate.get((accuracy, cost), 1))
                    for task in predicted_loads for accuracy in accuracy_levels for cost in cost_levels)

# Constraints

# Each task must have exactly one accuracy and cost level
for task in predicted_loads:
    model += pulp.lpSum(task_accuracy_cost[(task, accuracy, cost)] for accuracy in accuracy_levels for cost in cost_levels) == 1

# Predicted load on each node should not exceed the max load percentage
for node in current_loads:
    model += node_loads[node] <= max_load_percentage * nodes * current_loads[node]

# Predicted loads on nodes based on task assignments
for node in current_loads:
    model += node_loads[node] == pulp.lpSum(task_accuracy_cost[(task, accuracy, cost)] * predicted_loads[task][accuracy_levels.index(accuracy)]
                                             for task in predicted_loads for accuracy in accuracy_levels for cost in cost_levels)

# Solve the problem
model.solve()

# Output the results
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Total objective function value =", pulp.value(model.objective))

In [44]:
# Objective function: maximize accuracy, CPU utilization, and minimize cost
objective = pulp.lpSum([
    accuracy[i] * x[(i, k, n)] - cost[k] * x[(i, k, n)] - sla_violation_rate.get((i, k), 0) * x[(i, k, n)]
    for i in accuracy.keys() for k in cost.keys() for n in range(nodes)
])
model += objective

# Constraints
# Ensure total load does not exceed 80% of the total capacity per node at any timestamp
for t in range(6):
    for n in range(nodes):
        model += pulp.lpSum([
            x[(i, k, n)] * predicted_loads[c][t] 
            for i in accuracy.keys() for k in cost.keys() for c in predicted_loads.keys() if c == k * 2
        ]) <= max_load_percentage * sum(predicted_loads[c][t] for c in predicted_loads.keys())

# Each model should be assigned a core type and placed on exactly one node
for i in accuracy.keys():
    model += pulp.lpSum([x[(i, k, n)] for k in cost.keys() for n in range(nodes)]) == 1

# Solve the model
model.solve(pulp.PULP_CBC_CMD())

# Check if the model is feasible
if model.status == pulp.LpStatusInfeasible:
    print("Model is infeasible. Checking constraints.")
    # Print out the constraints to understand why they are infeasible
    for constraint in model.constraints.values():
        print(f"{constraint.name}: {constraint.value()}")

# Print the solution if it's optimal
if model.status == pulp.LpStatusOptimal:
    for i in accuracy.keys():
        for k in cost.keys():
            for n in range(nodes):
                if x[(i, k, n)].varValue > 0.5:
                    print(f"YOLO model {i} will be deployed on a {k}-core container on node {n}")
else:
    print("No optimal solution found.")


NameError: name 'accuracy' is not defined

In [ ]:
accuracy = {'nano': 45.7, 'small': 56.8, 'medium': 64.1}
cost = {0.5: 0.5, 1: 1, 2: 2}
predicted_loads = {
    2: [56.758137, 56.95575, 56.832123, 56.850445, 56.78143, 56.77969],
    4: [43.508926, 48.362938, 43.35024, 47.931683, 43.576538, 47.833996],
    6: [38.18736, 38.50672, 38.527256, 38.652214, 38.768574, 38.929024]
}
sla_violation_rate = {
    ('nano', 0.5): 0.05,
    ('small', 1): 0.09,
    ('small', 0.5): 0.3,
    ('medium', 2): 0
}
current_model_core_assignment = {
    'node0': ('nano', 0.5),
    'node1': ('small', 1),
    'node2': ('medium', 2)
}
current_cpu_utilization = [60, 50, 70]  # Current CPU utilization percentage for each node
nodes = 3  # Total number of nodes in the cluster
max_load_percentage = 0.8

# Create a new model
model = pulp.LpProblem("edge_computing", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("x", [(i, k, n) for i in accuracy.keys() for k in cost.keys() for n in range(nodes)], 0, 1, pulp.LpBinary)

# Objective function: maximize accuracy, CPU utilization, and minimize cost
objective = pulp.lpSum([
    accuracy[i] * x[(i, k, n)] - cost[k] * x[(i, k, n)] - sla_violation_rate.get((i, k), 0) * x[(i, k, n)]
    for i in accuracy.keys() for k in cost.keys() for n in range(nodes)
])
model += objective


# GAIKUBE Attempt

In [1]:
# Params:
"""
1. Predicetd load of each VM for 6 timestamps 
2. Current Utilization of each VM
3. VM remaining resources
4. Maximum Utilization limit of each VM
5. Do we need the Alloactaables?
6. Container can request for [0.5, 1, 2] core CPU
7. Coontainer can have ['nano', 'small', 'medium']
8. SLA violation rates of all containers (core, model) combinations
9. Model version to accuracy
10. Cost
11. Current container (core, model, node_id)
12. (Core, Model) processing time
13. 10% SLA violation vs 0% Violations
"""

"""
Objective:
    1. Maximize VM Utilizations
    2. Maximize Accuracy
    3. Minimize cost in terms of core
"""

'\nObjective:\n    1. Maximize VM Utilizations\n    2. Maximize Accuracy\n    3. Minimize cost in terms of core\n'

In [46]:
# index is vm core, value are next 6 timestamps CPU load predictions
from pulp import *
num_vms = 3
# index is VM id, value[0] is VM core, value[1] is a list of next 6 timestamp CPU utilization perdictions of each core in percentage
vm_predicted_utilization = {
    0: (2, [56.758137, 56.95575, 56.832123, 56.850445, 56.78143, 56.77969]),
    1: (4, [43.508926, 48.362938, 43.35024, 47.931683, 43.576538, 47.833996]),
    2: (6, [38.18736, 38.50672, 38.527256, 38.652214, 38.768574, 38.929024])
}
# current utilization of each VM in cluster
# index is VM_id, value[0] is vm core and value[1] is current utilization of this vm in percentage
vm_current_utilization = {
    0: (2, 25.15), 
    1: (4, 20.03),
    2: (6, 60.04)
}
# index is VM_id, value[0] is vm core and value[1] is the remaining CPU of each VM in cores
vm_cpu_remainining = {
    0: (2, 1.314),
    1: (4, 2.2), 
    2: (6,3.4)
    }
# maximum utilization allowed for each core vm
vm_max_util = {
    2: 80, 4:80, 6:80
}

vm_cap_requested = {
    0: (0.616, 1.93),
    1: (3.620, 3.92),
    2: (4.1, 5.8)
}

# total number of containers in the system
num_containers = 4

# cost is defined by the amount of cores used by containers requested from VMs. So, half means a cost of 0.5
container_cost = {
    'half':0.5, 
    'one':1, 
    'two':2
}
# machine learning model version and their associated accuracy
model_accuracy = {'nano': 45.7, 'small': 56.8, 'medium': 64.1}
# each container can have one selected core from below given pool
container_cores = [0.5, 1, 2]
# each container can have one selected model from below given list 
container_models = ['nano', 'small', 'medium']
# 

# index is contianer_id, value[0] is current model version, value[1] is current container core, value[2] is current SLA violation rate, value[3] is current VM id where this container is placed
container_details = {
    0: ('medium', 0.5, 1, 0),
    1: ('small', 1, 0.8, 2),
    2: ('small', 0.5, 0.2, 1),
    3: ('medium', 2, 1, 2)
}

# index is tuple of (model, container_core): value is processing time in milliseconds
container_delay= {
    ('nano', 0.5): 902.037,
    ('nano', 1): 499.33, 
    ('nano', 2): 368.76,
    ('small', 0.5): 2015.697, 
    ('small',1): 895.28,
    ('small', 2): 569.76,
    ('medium', 0.5): 6662.79,
    ('medium', 1): 2349.166,
    ('medium', 2): 925.28
}

In [48]:
cost

NameError: name 'cost' is not defined

In [50]:
prob = pulp.LpProblem("Container_Placement", LpMaximize)


# Decision variables
x = {(i, model, core): pulp.LpVariable(f"x_{i}_{model}_{core}", cat='Binary')
     for i in range(num_containers)
     for model in container_models
     for core in container_cores}

y = {(i, v): pulp.LpVariable(f"y_{i}_{v}", cat='Binary')
     for i in range(num_containers)
     for v in range(num_vms)}

# Objective function components
accuracy_sum = pulp.lpSum(model_accuracy[model] * x[(i, model, core)] for i in range(num_containers) for model in container_models for core in container_cores)
cost_sum = pulp.lpSum(container_cost['half' if core == 0.5 else 'one' if core == 1 else 'two'] * x[(i, model, core)] for i in range(num_containers) for model in container_models for core in container_cores)
sla_violations = pulp.lpSum(container_details[i][2] * x[(i, model, core)]
                            for i in range(num_containers)
                            for model in container_models
                            for core in container_cores
                            if model == container_details[i][0] and core == container_details[i][1])

# Objective function (maximize accuracy, minimize cost, minimize SLA violations)
prob += accuracy_sum - cost_sum - sla_violations, "Objective"


TypeError: 'float' object is not subscriptable

In [40]:
# Constraints
# Ensure each container selects exactly one model and one core
for i in range(num_containers):
    prob += pulp.lpSum(x[(i, model, core)] for model in container_models for core in container_cores) == 1, f"ModelCoreSelection_{i}"

# Ensure each container is placed on exactly one VM
for i in range(num_containers):
    prob += pulp.lpSum(y[(i, v)] for v in range(num_vms)) == 1, f"VMPlacement_{i}"

for v in range(num_vms):
    current_util = vm_current_utilization[v][1]
    max_capacity = vm_cap_requested[v][1]
    allowed_capacity = max_capacity*0.8
    vm_core = vm_current_utilization[v][0]
    current_requested = vm_cap_requested[v][0]
    #current_util = vm_current_utilization[v][1]
    prob += pulp.lpSum(y[(i, v)] * core for i in range(num_containers) for model in container_models for core in container_cores if x[(i, model, core)] == 1) + current_requested <= allowed_capacity, f"CPUUtilization_{v}"

# Additional constraints and conditions (such as SLA violation handling)
for i in range(num_containers):
    if container_details[i][2] >= 0.1:
        # Add constraint to select a lower accuracy model
        prob += pulp.lpSum(x[(i, model, core)] for model in container_models if model != 'medium' for core in container_cores) == 1, f"SLAViolationHigh_{i}"
    elif container_details[i][2] == 0:
        # Add constraint to select a higher accuracy model
        prob += pulp.lpSum(x[(i, model, core)] for model in container_models if model == 'medium' for core in container_cores) == 1, f"SLAViolationZero_{i}"

In [41]:
vm_cap_requested, vm_current_utilization, vm_predicted_utilization

({0: (0.616, 1.93), 1: (3.62, 3.92), 2: (4.1, 5.8)},
 {0: (2, 25.15), 1: (4, 20.03), 2: (6, 60.04)},
 {0: (2, [56.758137, 56.95575, 56.832123, 56.850445, 56.78143, 56.77969]),
  1: (4, [43.508926, 48.362938, 43.35024, 47.931683, 43.576538, 47.833996]),
  2: (6, [38.18736, 38.50672, 38.527256, 38.652214, 38.768574, 38.929024])})

In [42]:
# Solve the problem
status = prob.solve()
# Output the results
for v in range(num_vms):
    for i in range(num_containers):
        for model in container_models:
            for core in container_cores:
                if pulp.value(x[(i, model, core)]) == 1:
                    print(f"Container {i} is assigned model {model} with core {core} on VM {v}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/babarali/miniconda3/envs/bitbrain/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4ab815ffbdba45768f8a58374f733ebc-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/4ab815ffbdba45768f8a58374f733ebc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 237 RHS
At line 253 BOUNDS
At line 302 ENDATA
Problem MODEL has 15 rows, 48 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Container 0 is assigned model small with core 0.5 on VM 0
Container 0 is assigned model medium with core 1 on VM 0
Container 1 is assigned model small with core 0.5 on VM 0
Container 1 is assigned model medium with

In [43]:
LpStatus[status]

'Infeasible'